In [2]:
import pexpect

In [3]:
import json
import os
import subprocess
import tempfile
import time
import traceback
from pprint import pprint

HOME_DIR = os.path.expanduser('~')
DEFAULT_LAKE_PATH = f'{HOME_DIR}/.elan/bin/lake'
DEFAULT_LEAN_WORKSPACE = 'mathlib4/'

lake_path=DEFAULT_LAKE_PATH
lean_workspace=DEFAULT_LEAN_WORKSPACE

LEAN4_DEFAULT_HEADER = "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n"



In [4]:
full_code = [
    r'''import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

''',
    r'''/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
''',
    r'''  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
''',
  r'''  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
''',
    r'''  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
''',
    r'''  simpa [h₀] using h₄
'''
]

In [5]:
print("".join(full_code))

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄



In [7]:
child = pexpect.spawn(f"{lake_path} exe repl", cwd = lean_workspace)

In [8]:
TIMEOUT = 20
# 20-second timeout for the Lean4 REPL

In [9]:
def send_code_read_json(child: pexpect.spawn, cmd):
    cmd_json = json.dumps(cmd)
    print(cmd_json)
    child.send(cmd_json + "\r\n")
    # Read the input itself.
    # This should be printed instantly, so timeout is set to 1 second.
    child.expect_exact(cmd_json + "\r\n", timeout = 1)
    assert child.after.decode('utf-8') == cmd_json + "\r\n"

    res = ""
    # while res is not a valid json string, read lines.
    start_time = time.time()
    while True:
        try:
            res = res + child.readline().decode('utf-8')
            json.loads(res)
            break
        except json.JSONDecodeError:
            pass
        if time.time() - start_time > TIMEOUT:
            raise TimeoutError("Lean4 REPL timed out.")
    return json.loads(res)

In [12]:
times = []
for i in range(10):
    start = time.time()
    res = send_code_read_json(
        child,
        {
            "cmd": full_code[0],
            "allTactics": True,
            "tactics" : True
        }
    )
    times.append(time.time() - start)

mean = sum(times) / len(times)
stdev = (sum((x - mean) ** 2 for x in times) / len(times)) ** 0.5
print(f"Execution time: {mean:.2f} +- {stdev:.2f} seconds")
# Executes in 2.4 seconds.

{"cmd": "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\nset_option linter.unusedVariables false\nset_option linter.unnecessarySeqFocus false\n\nopen BigOperators Real Nat Topology Rat\n\n", "allTactics": true, "tactics": true}
{"cmd": "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\nset_option linter.unusedVariables false\nset_option linter.unnecessarySeqFocus false\n\nopen BigOperators Real Nat Topology Rat\n\n", "allTactics": true, "tactics": true}


In [1]:
res = send_code_read_json(
    child,
    {
        "cmd": full_code[0] + full_code[1]
        "allTactics": True,
        "tactics" : True
    }
)
res

NameError: name 'send_code_read_json' is not defined

In [ ]:
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',


{"cmd": "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\nset_option linter.unusedVariables false\nset_option linter.unnecessarySeqFocus false\n\nopen BigOperators Real Nat Topology Rat\n\n/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n  (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n  have h\u2081' : a * r = 2 := by simpa [h\u2080] using h\u2081\n  have h\u2082' : a * r ^ 3 = 6 := by simpa [h\u2080] using h\u2082\n  have h\u2083 : r ^ 2 = 3 := by\n    nlinarith\n  have h\u2084 : a = 2 / Real.sqrt 3 \u2228 a = -(2 / Real.sqrt 3) := by\n    apply eq_or_eq_neg_of_sq_eq_sq <;>\n    field_simp <;

{'tactics': [{'tactic': 'simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero, Nat.succ_add]',
   'proofState': 0,
   'pos': {'line': 14, 'column': 2},
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
   'endPos': {'line': 15, 'column': 17}},
  {'tactic': "have h₁' : a * r = 2 := by simpa [h₀] using h₁",
   'proofState': 1,
   'pos': {'line': 16, 'column': 2},
   'goals': 'a r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)',
   'endPos': {'line': 16, 'column': 48}},
  {'tactic': 'simpa [h₀] using h₁',
   'proofState': 2,
   'pos': {'line': 16, 'column': 29},
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : a * r ^ succ 0 = 2 h₂ : a * r ^ 3 = 6 ⊢ a * r = 2',
   'endPos': {'line': 16, 'column': 48}},
  {'tactic': "have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂",
   'p

In [287]:
res = send_code_read_json(
    child,
    {
        "cmd": full_code[2],
        "env" : 0
    }
)
res

{"cmd": "  have h\u2081' : a * r = 2 := by simpa [h\u2080] using h\u2081\n  have h\u2082' : a * r ^ 3 = 6 := by simpa [h\u2080] using h\u2082\n  have h\u2083 : r ^ 2 = 3 := by\n    nlinarith\n", "env": 0}


{'messages': [{'severity': 'error',
   'pos': {'line': 1, 'column': 2},
   'endPos': {'line': 1, 'column': 6},
   'data': "unexpected token 'have'; expected command"}],
 'env': 1,
 'ast': []}

In [276]:
res = send_code_read_json(
    child,
    {
        "tactic": full_code[1],
        "proofState" : 0
    }
)
res

{"tactic": "  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n", "proofState": 0}


{'proofState': 2,
 'goals': ['a r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)']}

In [278]:
res = send_code_read_json(
    child,
    {
        "tactic": full_code[2],
        "proofState" : 2
    }
)
res

{"tactic": "  have h\u2081' : a * r = 2 := by simpa [h\u2080] using h\u2081\n  have h\u2082' : a * r ^ 3 = 6 := by simpa [h\u2080] using h\u2082\n  have h\u2083 : r ^ 2 = 3 := by\n    nlinarith\n", "proofState": 2}


{'message': 'Lean error:\n<input>:2:2: expected end of input'}